In [1]:
pwd

'C:\\Users\\Seo\\Desktop\\JaeHee\\Programming\\final_project\\강사님자료'

In [1]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math

from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [82]:
# 영화배우 이름 검색
name = '혜리'

# 크롬 옵션
options = webdriver.ChromeOptions()  

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800,1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(3)
driver.get("https://www.naver.com")

# 네이버 검색어 입력 후 검색
element = driver.find_element_by_name("query")
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(name)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(1)

# 프로필 클릭
driver.find_element_by_link_text("프로필").click()
time.sleep(1)

# 1. 배우 수상내역 수집하기

In [83]:
# 수상내역 더보기 클릭
try:
    driver.find_element_by_css_selector(".cm_content_area._cm_content_area_prize .area_button_arrow").click()
    time.sleep(1)
except:
    pass

In [84]:
# 수상내역 크롤링
prize_list = []

In [85]:
prize = driver.find_element_by_css_selector(".cm_content_area._cm_content_area_prize").text
prize

'수상내역\n2016\nSAF 연기대상 뉴스타상\n2016\n제1회 tvN10 어워즈 여자부문 대세배우상\n2016\n제5회 아시아태평양 스타어워즈 신인상\n2014\nMBC 방송연예대상 여자 신인상'

In [86]:
# 년도만 세기
p = re.compile('[0-9]{4,4}')  # 정규표현식 - 숫자 4자리만 추출하라
m = p.findall(prize)
m

['2016', '2016', '2016', '2014']

In [87]:
prize_list = prize_list + m
prize_list

['2016', '2016', '2016', '2014']

In [88]:
# 수상내역 다음페이지 클릭
if driver.find_element_by_css_selector('.pg_next.on'):
    next = driver.find_element_by_css_selector('.pg_next.on').click()
    time.sleep(2)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pg_next.on"}
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F7A8E3+2402531]
	Ordinal0 [0x00F0B011+1945617]
	Ordinal0 [0x00DFC748+837448]
	Ordinal0 [0x00E29330+1020720]
	Ordinal0 [0x00E295CB+1021387]
	Ordinal0 [0x00E563B2+1205170]
	Ordinal0 [0x00E442F4+1131252]
	Ordinal0 [0x00E546C2+1197762]
	Ordinal0 [0x00E440C6+1130694]
	Ordinal0 [0x00E1E676+976502]
	Ordinal0 [0x00E1F586+980358]
	GetHandleVerifier [0x011EBC42+2511938]
	GetHandleVerifier [0x011DED6F+2458991]
	GetHandleVerifier [0x010131AA+575914]
	GetHandleVerifier [0x01011FA6+571302]
	Ordinal0 [0x00F11DEB+1973739]
	Ordinal0 [0x00F16778+1992568]
	Ordinal0 [0x00F16865+1992805]
	Ordinal0 [0x00F1FA41+2030145]
	BaseThreadInitThunk [0x75F8FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77A07A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77A07A4E+238]


# 배우 수상내역 페이지 for문으로 묶기

In [2]:
# 영화배우 이름 검색
name = '혜리'

# 크롬 옵션
options = webdriver.ChromeOptions()  

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800,1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(3)
driver.get("https://www.naver.com")

# 네이버 검색어 입력 후 검색
element = driver.find_element_by_name("query")
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(name)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(1)

# 프로필 클릭
driver.find_element_by_link_text("프로필").click()
time.sleep(1)

# 수상내역 더보기 클릭
try:
    driver.find_element_by_css_selector(".cm_content_area._cm_content_area_prize .area_button_arrow").click()
    time.sleep(1)
except:
    pass

# 수상내역 갯수 세기
prize_list = []

for i in range(0, 100):  # 수상내역 페이지 최대 100페이지까지 수집할 수 있게 설정
    try:
        # 수상내역 크롤링
        prize = driver.find_element_by_css_selector(".cm_content_area._cm_content_area_prize").text
        # 년도만 세기
        p = re.compile('[0-9]{4,4}')   # 정규표현식으로 "0-9숫자 4자리" 수집
        m = p.findall(prize)           # prize 중에서 모두 찾아라(findall 함수)
        prize_list = prize_list + m    # 페이지 별 수상내역 리스트 전부 합치기

        # 수상내역 다음페이지 클릭
        if driver.find_element_by_css_selector('.pg_next.on'):
            next = driver.find_element_by_css_selector('.pg_next.on').click()
            time.sleep(2)
    except:  
        break
    
print('수상 수: ', len(prize_list))
prize_list

수상 수:  4


['2016', '2016', '2016', '2014']

# 2. 배우 필모그래피 수집하기

In [3]:
# 필모그래피 클릭
driver.find_element_by_link_text("필모그래피").click()
time.sleep(1)

In [4]:
# 인기순 클릭
driver.find_element_by_link_text("인기순").click()
time.sleep(1)

In [5]:
driver.find_element_by_css_selector('.this_text_number').text

'2'

In [6]:
film_num = int(driver.find_element_by_css_selector('.this_text_number').text)
print('필모 수: ', film_num)

필모 수:  2


In [7]:
# 페이지 클릭횟수 계산
page_click = math.ceil(film_num/4) - 1
print('페이지 클릭 횟수: ', page_click)

페이지 클릭 횟수:  0


In [8]:
# 영화 데이터 수집
title_list2 = []
score_list2 = []
cast_list2 = []
audiance_list2 = []
main_sub_list = []

In [9]:
i = 0

# 제목 수집 해보기

# 제목 페이지별 리스트
temp_list = []

titles = driver.find_elements_by_css_selector('.this_text')
title = titles[0]
print(title.text)

물괴


In [10]:
temp_list = []
for title in titles:
    temp_list.append(title.text)
    temp_list = [x for x in temp_list if x !='' and x != '상영중']
#     print(temp_list)    
title_list2 = title_list2 + temp_list    
title_list2

['물괴', '판소리 복서']

In [11]:
cast_temp = []
for t in range(1, 5):
    try:
        cast = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > span'.format(i+1, t)).text[:2]
        cast_temp.append(cast)
    except:
        break
cast_list2 = cast_list2 + cast_temp  

In [13]:
score_temp = []
for j in range(1, 5):
    try:
        score = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > div.info > dl:nth-child(2)'.format(i+1, j)).text.split('\n')[1]
        score = score.replace('만','0000').replace(',','')
        if '.' in score:
            score = int(score.replace('.', ''))
            score = round(score*0.1)
        else:
            score = round(int(score))

        score_temp.append(score)
    except:
        break
score_list2 = score_list2 + score_temp

['주연', '주연']

In [ ]:
title_list2 = title_list2[:len(score_list2)]
cast_list2 = cast_list2[:len(score_list2)]

    # print('\n', len(title_list))
    # print(title_list)
    # print(cast_list)
    # print(score_list)

df2 = pd.DataFrame(list(zip(title_list2, cast_list2, score_list2)), columns = ['제목','배역', 'score'])
print('관객수 데이터가 있는 영화들만: ')
print(d2f)

# 주연 스코어 합
cast_main = df2[(df2['배역']=='주연')]
main_score = cast_main['score'].sum()

# 조연 스코어 합
cast_sub = df2[(df2['배역']=='조연')]
sub_score = cast_sub['score'].sum()

# 단역 스코어 합
cast_other = df2[(df2['배역']!='주연') & (df2['배역']!='조연')]
cast_score = cast_other['score'].sum()

# 배우 흥행지수
actor_score_index = round((main_score*0.5 + sub_score*0.4 + cast_score*0.1) / len(df))
print('{} 배우 흥행지수: '.format(name), actor_score_index)

## 배우 필모그래피 for문으로 수집하기

In [ ]:
# 배우 리스트에서 이름 불러와서 name 변수에 저장하고 필모그래피 크롤링하기
# pd.read_csv("actor_list.csv")

In [14]:
import pandas as pd
data = pd.read_csv('data_star_기대.csv')

In [15]:
actor = data['actors'][0].split(',')
actor = actor[:5]

In [11]:
actor[1]

' 김인권'

In [16]:
# 영화배우 이름 검색
# 영화 데이터 수집
total_score = []

for i in range(len(data)) :   
    actor = data['actors'][i].split(',')
    actor = actor[:5]
    actor_score = []
    for ind in actor:    
        name = ind
        title_list = []
        score_list = []
        cast_list = []
        audiance_list = []
        main_sub_list = []
        # 크롬 옵션
        options = webdriver.ChromeOptions()

        # 크롬 윈도우 사이즈 조절
        options.add_argument("--window-size=800,1200") # window-size -> 기본 : 1920,1080

        import chromedriver_autoinstaller
        chrome_path = chromedriver_autoinstaller.install()
        driver = webdriver.Chrome(chrome_path, options=options)
        time.sleep(3)
        driver.get("https://www.naver.com")

        # 네이버 검색어 입력 후 검색
        element = driver.find_element_by_name("query")
        element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

        element.send_keys(name)  # 검색창에 검색어 전달
        element.submit()  # 검색 클릭
        time.sleep(1)

        try:
            # 필모그래피 클릭
            driver.find_element_by_link_text("필모그래피").click()
            time.sleep(1)

            # 인기순 클릭
            driver.find_element_by_link_text("인기순").click()
            time.sleep(1)

            film_num = int(driver.find_element_by_css_selector('.this_text_number').text)
            print('필모 수: ', film_num)

            # 페이지 클릭횟수 계산
            page_click = math.ceil(film_num/4) - 1
            print('페이지 클릭 횟수: ', page_click)


            for i in range(page_click):    
                # 제목
                titles = driver.find_elements_by_css_selector('.this_text')
                temp_list = []
                for title in titles:
                    temp_list.append(title.text)
                    temp_list = [x for x in temp_list if x !='' and x != '상영중']
            #     print(temp_list)    
                title_list = title_list + temp_list    

                # 배역
                cast_temp = []
                for t in range(1, 5):
                    try:
                        cast = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > span'.format(i+1, t)).text[:2]
                        cast_temp.append(cast)
                    except:
                        break
                cast_list = cast_list + cast_temp        
            #     print(cast_temp)


                # 관객수
                score_temp = []
                for j in range(1, 5):
                    try:
                        score = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > div.info > dl:nth-child(2)'.format(i+1, j)).text.split('\n')[1]
                        score = score.replace('만','0000').replace(',','')
                        if '.' in score:
                            score = int(score.replace('.', ''))
                            score = round(score*0.1)
                        else:
                            score = round(int(score))

                        score_temp.append(score)
                    except:
                        break
                score_list = score_list + score_temp
            #     print(score_temp, '\n')
                if len(score_temp) < 4:
                    break

                # 필모 다음페이지 클릭
                driver.find_element_by_css_selector('.pg_next.on').click()
                time.sleep(2)

        
            title_list = title_list[:len(score_list)]
            cast_list = cast_list[:len(score_list)]

                # print('\n', len(title_list))
                # print(title_list)
                # print(cast_list)
                # print(score_list)

            df = pd.DataFrame(list(zip(title_list, cast_list, score_list)), columns = ['제목','배역', 'score'])
            print('관객수 데이터가 있는 영화들만: ')
            print(df)

            # 주연 스코어 합
            cast_main = df[(df['배역']=='주연')]
            main_score = cast_main['score'].sum()

            # 조연 스코어 합
            cast_sub = df[(df['배역']=='조연')]
            sub_score = cast_sub['score'].sum()

            # 단역 스코어 합
            cast_other = df[(df['배역']!='주연') & (df['배역']!='조연')]
            cast_score = cast_other['score'].sum()

            # 배우 흥행지수
            actor_score_index = round((main_score*0.5 + sub_score*0.4 + cast_score*0.1) / len(df))
            print('{} 배우 흥행지수: '.format(name), actor_score_index)
            actor_score.append(actor_score_index)
        except:
            actor_score.append(0)
    total_score.append(sum(actor_score))

필모 수:  19
페이지 클릭 횟수:  4
관객수 데이터가 있는 영화들만: 
                  제목  배역    score
0   조선명탐정: 각시투구꽃의 비밀  주연  4780000
1                판도라  특별  4580000
2                연가시  주연  4510000
3   조선명탐정 : 사라진 놉의 딸  주연  3870000
4    조선명탐정: 흡혈괴마의 비밀  주연  2440000
5          내 사랑 내 곁에  주연  2130000
6             무방비 도시  주연  1600000
7              브이아이피  주연  1370000
8                 간첩  주연  1310000
9      특별수사: 사형수의 편지  주연  1240000
10     장사리 : 잊혀진 영웅들  주연  1140000
11                하루  주연  1120000
12           파괴된 사나이  주연  1010000
13                물괴  주연   720000
14                리턴  주연   640000
15           페이스 메이커  주연   460000
김명민 배우 흥행지수:  914250
필모 수:  49
페이지 클릭 횟수:  12
관객수 데이터가 있는 영화들만: 
               제목  배역     score
0     광해, 왕이 된 남자  조연  12320000
1             해운대  조연  11320000
2            히말라야  조연   7750000
3              타워  조연   5180000
4         타짜-신의 손  조연   4010000
5          신의 한 수  주연   3560000
6               퀵  주연   3120000
7            마이웨이  조연   2140000
8            초능력자  특별   21

필모 수:  19
페이지 클릭 횟수:  4
관객수 데이터가 있는 영화들만: 
             제목  배역    score
0           아가씨  주연  4280000
1            화차  주연  2430000
2        연애의 온도  주연  1860000
3         우는 남자  주연   600000
4     뜨거운 것이 좋아  주연   570000
5          여배우들  주연   510000
6           모비딕  주연   430000
7   지금은맞고그때는틀리다  주연    81000
8    밤의 해변에서 혼자  주연    57000
9           그 후  조연    19000
10       도망친 여자  주연    10000
11     클레어의 카메라  주연     9439
12      소설가의 영화  주연     8224
13          풀잎들  주연     7461
14         강변호텔  조연     6975
15        인트로덕션  주연     6214
김민희 배우 흥행지수:  340004
필모 수:  112
페이지 클릭 횟수:  27
관객수 데이터가 있는 영화들만: 
                                   제목  배역    score
0                                 군함도  단역  6590000
1                                  공작  조연  4970000
2                                 용의자  조연  4130000
3                              끝까지 간다  우정  3450000
4                                 친구2  조연  2970000
5                                 몽타주  조연  2090000
6                              봉이 김선달  조연

필모 수:  47
페이지 클릭 횟수:  11
관객수 데이터가 있는 영화들만: 
                제목  배역    score
0             1987  특별  7230000
1              아가씨  특별  4280000
2     우리 생애 최고의 순간  주연  4010000
3              스파이  주연  3430000
4               하녀  조연  2260000
5        마당을 나온 암탉  주연  2200000
6          리틀 포레스트  주연  1500000
7             특별시민  주연  1360000
8           관능의 법칙  주연   780000
9          효자동 이발사  주연   740000
10     여교수의 은밀한 매력  주연   600000
11          가려진 시간  특별   510000
12        사랑해, 말순씨  주연   340000
13  니 부모 얼굴이 보고 싶다  조연   330000
14            배심원들  주연   280000
15         분노의 윤리학  주연   220000
16          가족의 탄생  주연   200000
17             세자매  주연    84000
18              사과  주연    58000
19             하하하  주연    57000
20           작은 연못  단역    46000
21              메기  주연    42000
22      잘 알지도 못하면서  단역    40000
23          자유의 언덕  주연    39000
24              만신  주연    37000
25          다른나라에서  조연    31000
26          날아라 펭귄  주연    20000
27        여배우는 오늘도  감독    17000
28    군산: 거위를 노래하다  주연    15

필모 수:  12
페이지 클릭 횟수:  2
관객수 데이터가 있는 영화들만: 
            제목  배역    score
0         청년경찰  특별  5650000
1   지금 만나러 갑니다  조연  2600000
2  타짜: 원 아이드 잭  단역  2220000
3           사자  단역  1610000
4        동네사람들  특별   460000
5         배심원들  조연   280000
6       어린 의뢰인  조연   200000
7       당신의 부탁  조연    26000
 서정연 배우 흥행지수:  279550
필모 수:  37
페이지 클릭 횟수:  9
관객수 데이터가 있는 영화들만: 
                 제목  배역    score
0              검사외전  단역  9700000
1      나쁜 녀석들: 더 무비  단역  4570000
2           거북이 달린다  단역  3020000
3           헬로우 고스트  단역  3010000
4            님은 먼곳에  단역  1700000
5         나의 특별한 형제  단역  1470000
6      찌라시 : 위험한 소문  조연  1220000
7           육혈포 강도단  단역  1210000
8               황진이  단역  1190000
9           슬로우 비디오  단역  1160000
10               간신  단역  1110000
11          남쪽으로 튀어  단역   830000
12           김씨 표류기  단역   720000
13               창수  단역   420000
14          사랑이 무서워  단역   400000
15           나의 독재자  단역   380000
16      누가 그녀와 잤을까?  단역   350000
17            소셜포비아  단역   250000
18          방

필모 수:  45
페이지 클릭 횟수:  11
관객수 데이터가 있는 영화들만: 
                   제목  배역    score
0                 전우치  조연  6060000
1                  타짜  조연  5690000
2                 완득이  조연  5310000
3                  이끼  주연  3350000
4                  식객  조연  2960000
5                  소원  조연  2710000
6             너는 내 운명  조연  2700000
7                 마파도  특별  2630000
8                 목격자  주연  2520000
9              조작된 도시  조연  2510000
10            뷰티 인사이드  주연  2050000
11                 협상  조연  1960000
12                 대호  조연  1760000
13               잠복근무  조연  1530000
14                 해무  주연  1470000
15                런닝맨  주연  1420000
16        구르믈 버서난 달처럼  우정  1380000
17                각설탕  조연  1260000
18      특별수사: 사형수의 편지  주연  1240000
19             즐거운 인생  주연  1200000
20       당신, 거기 있어줄래요  조연  1160000
21             심장이 뛴다  조연  1020000
22             미쓰 와이프  조연   980000
23            범죄의 재구성  조연   940000
24            그때 그사람들  조연   890000
25  연애, 그 참을 수 없는 가벼움  조연   560000
26         

...

필모 수:  52
페이지 클릭 횟수:  12
관객수 데이터가 있는 영화들만: 
                   제목  배역     score
0                  괴물  조연  10910000
1   좋은 놈, 나쁜 놈, 이상한 놈  조연   6680000
2                덕혜옹주  조연   5590000
3                   퀵  조연   3120000
4                  마더  조연   2980000
5              그놈 목소리  우정   2970000
6             너는 내 운명  조연   2700000
7                 아수라  특별   2590000
8         타짜: 원 아이드 잭  조연   2220000
9         천문: 하늘에 묻는다  조연   2000000
10             그림자 살인  조연   1890000
11             귀신이 산다  단역   1870000
12                마약왕  조연   1860000
13             비열한 거리  조연   1820000
14                 차우  주연   1770000
15             전설의 주먹  주연   1740000
16                평양성  주연   1710000
17                 해무  특별   1470000
18             고령화 가족  주연   1140000
19                동창생  주연   1040000
20             우아한 세계  조연    960000
21               남극일기  조연    870000
22               상류사회  조연    760000
23              아빠는 딸  주연    640000
24             널 기다리며  주연    630000
25   지푸라기라도 잡고 싶은 짐승

필모 수:  17
페이지 클릭 횟수:  4
관객수 데이터가 있는 영화들만: 
           제목  배역    score
0      수상한 그녀  단역  8660000
1         더 킹  조연  5310000
2      오싹한 연애  조연  3000000
3      굿바이 싱글  우정  2100000
4   나의 PS 파트너  조연  1830000
5     슬로우 비디오  단역  1160000
6      여고괴담 5  조연   640000
7        요가학원  조연   260000
8          부산  단역   150000
9         족구왕  주연    48000
10     범죄의 여왕  우정    44000
11  우리 연애의 이력  조연     3043
12      메멘토모리  주연      151
 황승언 배우 흥행지수:  434823
필모 수:  2
페이지 클릭 횟수:  0
관객수 데이터가 있는 영화들만: 
Empty DataFrame
Columns: [제목, 배역, score]
Index: []
필모 수:  11
페이지 클릭 횟수:  2
관객수 데이터가 있는 영화들만: 
            제목  배역   score
0         돌연변이  조연  100000
1          족구왕  주연   48000
2       범죄의 여왕  우정   44000
3  이것이 우리의 끝이다  조연    2941
4            못  주연    2386
5           들꽃  주연    1295
6          캐논볼  조연     413
 강봉성 배우 흥행지수:  10226
필모 수:  17
페이지 클릭 횟수:  4
관객수 데이터가 있는 영화들만: 
           제목  배역    score
0          스물  단역  3040000
1      굿바이 싱글  조연  2100000
2    바람 바람 바람  단역  1190000
3         소공녀  단역    60000
4     

필모 수:  40
페이지 클릭 횟수:  9
관객수 데이터가 있는 영화들만: 
Empty DataFrame
Columns: [제목, 배역, score]
Index: []
필모 수:  54
페이지 클릭 횟수:  13
관객수 데이터가 있는 영화들만: 
                    제목  배역     score
0                  변호인  조연  11370000
1                 검사외전  조연   9700000
2    좋은 놈, 나쁜 놈, 이상한 놈  단역   6680000
3                   공작  주연   4970000
4            군도:민란의 시대  조연   4770000
5              남산의 부장들  주연   4750000
6           내 아내의 모든 것  우정   4590000
7                  말아톤  단역   4190000
8                 부당거래  조연   2720000
9                  보안관  주연   2580000
10                 목격자  주연   2520000
11                블랙머니  특별   2480000
12              굿바이 싱글  특별   2100000
13                 마약왕  조연   1860000
14                 해결사  조연   1840000
15                  뺑반  조연   1820000
16                  인질  우정   1630000
17           두근두근 내 인생  특별   1620000
18                 하울링  조연   1610000
19                  밀양  단역   1600000
20            바람 바람 바람  주연   1190000
21                 빅매치  주연   1170000
22         

필모 수:  44
페이지 클릭 횟수:  10
관객수 데이터가 있는 영화들만: 
                     제목  배역     score
0             신과함께-죄와 벌  주연  14410000
1                    암살  주연  12700000
2             신과함께-인과 연  주연  12270000
3                  국가대표  주연   8390000
4                   백두산  주연   8250000
5                  1987  주연   7230000
6                   베를린  주연   7160000
7                    터널  주연   7120000
8              더 테러 라이브  주연   5580000
9                   추격자  주연   5040000
10            군도:민란의 시대  주연   4770000
11  범죄와의 전쟁 : 나쁜놈들 전성시대  주연   4720000
12                  아가씨  주연   4280000
13         우리 생애 최고의 순간  조연   4010000
14                  의뢰인  주연   2390000
15                   황해  주연   2260000
16                 러브픽션  주연   1720000
17            PMC: 더 벙커  주연   1670000
18                  걸캅스  단역   1620000
19                 잠복근무  주연   1530000
20                  클로젯  주연   1270000
21                  허삼관  감독    950000
22                 평행이론  조연    920000
23              비스티 보이즈  주연    710000
24    

필모 수:  32
페이지 클릭 횟수:  7
관객수 데이터가 있는 영화들만: 
                  제목  배역     score
0                도둑들  주연  12980000
1               1987  주연   7230000
2                전우치  주연   6060000
3                 타짜  조연   5690000
4             검은 사제들  주연   5440000
5                완득이  주연   5310000
6                추격자  주연   5040000
7            타짜-신의 손  조연   4010000
8               남한산성  주연   3840000
9               암수살인  주연   3780000
10              모가디슈  주연   3610000
11           거북이 달린다  주연   3020000
12              극비수사  주연   2860000
13    화이 : 괴물을 삼킨 아이  주연   2390000
14  내 생애 가장 아름다운 일주일  단역   2290000
15                황해  주연   2260000
16               쎄시봉  주연   1710000
17                해무  주연   1470000
18           시실리 2km  조연   1260000
19            즐거운 인생  주연   1200000
20      당신, 거기 있어줄래요  주연   1160000
21           범죄의 재구성  조연    940000
22                야수  조연    830000
23           남쪽으로 튀어  주연    830000
24          천하장사 마돈나  조연    570000
25               미성년  감독    290000
26          

In [17]:
len(total_score)

575

In [23]:
score = pd.DataFrame({'filmography':total_score})

In [24]:
score

,filmography
0,3859676
1,3417714
2,3051601
3,872345
4,2363703
...,...
570,3695032
571,3039306
572,4383350
573,777622


In [25]:
score.to_csv('filmography.csv', index=False)

In [18]:
title_list = []
score_list = []
cast_list = []
audiance_list = []
main_sub_list = []
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800,1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(3)
driver.get("https://www.naver.com")

# 네이버 검색어 입력 후 검색
element = driver.find_element_by_name("query")
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys('혜리')  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(1)

try:
    # 필모그래피 클릭
    driver.find_element_by_link_text("필모그래피").click()
    time.sleep(1)

    # 인기순 클릭
    driver.find_element_by_link_text("인기순").click()
    time.sleep(1)

    film_num = int(driver.find_element_by_css_selector('.this_text_number').text)
    print('필모 수: ', film_num)

    # 페이지 클릭횟수 계산
    page_click = math.ceil(film_num/4) - 1
    print('페이지 클릭 횟수: ', page_click)

    
    for i in range(page_click):    
        # 제목
        titles = driver.find_elements_by_css_selector('.this_text')
        temp_list = []
        for title in titles:
            temp_list.append(title.text)
            temp_list = [x for x in temp_list if x !='' and x != '상영중']
    #     print(temp_list)       
        title_list = title_list + temp_list 
        # 배역
        
        cast_temp = []
        for t in range(1, 5):
            try:
                cast = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > span'.format(i+1, t)).text[:2]
                cast_temp.append(cast)
            except:
                break
        cast_list = cast_list + cast_temp        
    #     print(cast_temp)
        
        # 관객수
        score_temp = []
        for j in range(1, 5):
            try:
                score = driver.find_element_by_css_selector('#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > div.info > dl:nth-child(2)'.format(i+1, j)).text.split('\n')[1]
                score = score.replace('만','0000').replace(',','')
                if '.' in score:
                    score = int(score.replace('.', ''))
                    score = round(score*0.1)
                else:
                    score = round(int(score))

                score_temp.append(score)
            except:
                break
        score_list = score_list + score_temp
    #     print(score_temp, '\n')
        if len(score_temp) < 4:
            break

        # 필모 다음페이지 클릭
        driver.find_element_by_css_selector('.pg_next.on').click()
        time.sleep(2)
except:
    pass

title_list = title_list[:len(score_list)]
cast_list = cast_list[:len(score_list)]

    # print('\n', len(title_list))
    # print(title_list)
    # print(cast_list)
    # print(score_list)

df = pd.DataFrame(list(zip(title_list, cast_list, score_list)), columns = ['제목','배역', 'score'])
print('관객수 데이터가 있는 영화들만: ')
print(df)

# 주연 스코어 합
cast_main = df[(df['배역']=='주연')]
main_score = cast_main['score'].sum()

# 조연 스코어 합
cast_sub = df[(df['배역']=='조연')]
sub_score = cast_sub['score'].sum()

# 단역 스코어 합
cast_other = df[(df['배역']!='주연') & (df['배역']!='조연')]
cast_score = cast_other['score'].sum()

# 배우 흥행지수
actor_score_index = round((main_score*0.5 + sub_score*0.4 + cast_score*0.1) / len(df))
print('{} 배우 흥행지수: '.format(name), actor_score_index)

필모 수:  2
페이지 클릭 횟수:  0
관객수 데이터가 있는 영화들만: 
Empty DataFrame
Columns: [제목, 배역, score]
Index: []


ZeroDivisionError: float division by zero

In [19]:
title_list

[]

['물괴', '판소리 복서']

In [4]:
cast_main = df[(df['배역']=='주연')]
cast_main

,제목,배역,score
2,모가디슈,주연,3610000
4,메기,주연,42000
5,꿈의 제인,주연,25000
6,황금시대,주연,3792
8,우리 손자 베스트,주연,2327
9,서울연애,주연,1547
